In [1]:
from copy import deepcopy
import subprocess
import numpy as np
import open3d as o3d
import trimesh
import os
print(o3d.__version__)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
0.18.0


In [2]:
def subdivide_surface_fitting(decimated_mesh, target_mesh, iterations=1):
    subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_mesh, number_of_iterations=iterations)
    print(subdivided_mesh)
    subdivided_mesh.compute_vertex_normals()
    
    pcd_target = o3d.geometry.PointCloud()
    pcd_target.points = o3d.utility.Vector3dVector(target_mesh.vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_target)
    subdivided_vertices = np.array(subdivided_mesh.vertices)
    target_vertices = np.array(target_mesh.vertices)
    fitting_vertices = deepcopy(subdivided_vertices)
    
    for i in range(0, len(subdivided_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_vertices[i], 1)
        fitting_vertices[i] = target_vertices[index[0]]

    subdivided_mesh.vertices = o3d.utility.Vector3dVector(fitting_vertices)
    return subdivided_mesh

In [3]:
def read_triangle_mesh_with_trimesh(avatar_name,enable_post_processing=False):
    # EDIT: next 4 lines replace to maintain order even in case of degenerate and non referenced
    # scene_patch = trimesh.load(avatar_name,process=enable_post_processing)
    if enable_post_processing:
        scene_patch = trimesh.load(avatar_name,process=True)
    else:
        scene_patch = trimesh.load(avatar_name,process=False,maintain_order=True) 
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(scene_patch.vertices),
        o3d.utility.Vector3iVector(scene_patch.faces)
    ) 
    if scene_patch.vertex_normals.size:
        mesh.vertex_normals = o3d.utility.Vector3dVector(scene_patch.vertex_normals.copy())
    if scene_patch.visual.defined:
        # either texture or vertex colors if no uvs present.
        if scene_patch.visual.kind == 'vertex':
            mesh.vertex_colors = o3d.utility.Vector3dVector(scene_patch.visual.vertex_colors[:,:3]/255) # no alpha channel support
        elif scene_patch.visual.kind == 'texture':
            uv = scene_patch.visual.uv
            if uv.shape[0] == scene_patch.vertices.shape[0]:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv[scene_patch.faces.flatten()])
            elif uv.shape[0] != scene_patch.faces.shape[0] * 3:
                assert False
            else:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv)
                if scene_patch.visual.material is not None and scene_patch.visual.material.image is not None:
                    if scene_patch.visual.material.image.mode == 'RGB':
                        mesh.textures = [o3d.geometry.Image(np.asarray(scene_patch.visual.material.image))]
                    else:
                        assert False
        else:
            assert False
    return mesh

In [5]:
load_reference_mesh = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj', enable_post_processing = False)
print(load_reference_mesh)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj', load_reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

TriangleMesh with 20002 points and 40000 triangles.


True

In [7]:
load_reference_mesh = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj', enable_post_processing = False)
print(np.array(load_reference_mesh.vertices))

[[ 0.0791106   0.90974599 -0.477777  ]
 [ 0.0861452   0.90974599 -0.48245999]
 [ 0.0861452   0.905756   -0.48109001]
 ...
 [-0.127333    1.02546     0.26905501]
 [ 0.174541   -0.25041699  0.27527499]
 [-0.0575035   0.39899501  0.31645399]]


In [90]:
load_reference_mesh_trimesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj', enable_post_processing = False)

print(np.array(load_reference_mesh_trimesh.vertices).__len__())
decimated_reference_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(load_reference_mesh_trimesh, 10000, boundary_weight= 8000)
print(decimated_reference_mesh)

subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_reference_mesh, number_of_iterations=1)
print(subdivided_mesh)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', decimated_reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/subdivided_reference_mesh.obj', subdivided_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)


20002
TriangleMesh with 5002 points and 10000 triangles.
TriangleMesh with 20002 points and 40000 triangles.


True

In [74]:
subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_reference_mesh, number_of_iterations=1)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/subdivided_reference_mesh1.obj', subdivided_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

True

In [91]:
reconstruct_dancer_5 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_005.obj')
original_dancer_5 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0005.obj')
print(reconstruct_dancer_5)
print(original_dancer_5)
reconstruct_dancer_5.compute_vertex_normals()
original_dancer_5.compute_vertex_normals()
o3d.visualization.draw_geometries([reconstruct_dancer_5])

TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 19724 points and 39380 triangles.


In [8]:
decimated_reconstructed_5 = o3d.geometry.TriangleMesh.simplify_quadric_decimation(reconstruct_dancer_5, 10000, boundary_weight= 8000)
fitting_mesh_dancer_5 = subdivide_surface_fitting(decimated_reconstructed_5, original_dancer_5, 1)
fitting_mesh_dancer_5.compute_vertex_normals()

o3d.visualization.draw_geometries([fitting_mesh_dancer_5])


TriangleMesh with 5002 points and 10000 triangles.
TriangleMesh with 20002 points and 40000 triangles.
TriangleMesh with 20002 points and 40000 triangles.


In [ ]:
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_5.obj', fitting_mesh_dancer_5, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

In [5]:
GoF = 5
for i in range(0, GoF):
    offset = 5
    reconstruct_dancer_i = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_{i+offset:03}.obj')
    original_dancer_i = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0{i+offset:03}.obj')
    #print(reconstruct_dancer_i)
    #print(original_dancer_i)
    reconstruct_dancer_i.compute_vertex_normals()
    original_dancer_i.compute_vertex_normals()
    #o3d.visualization.draw_geometries([reconstruct_dancer_i])
    decimated_reconstructed_i = o3d.geometry.TriangleMesh.simplify_quadric_decimation(reconstruct_dancer_i, 10000, boundary_weight= 8000)
    #print(decimated_reconstructed_i)
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/decimated_reconstructed_{i+offset:03}.obj', decimated_reconstructed_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    fitting_mesh_dancer_i = subdivide_surface_fitting(decimated_reconstructed_i, original_dancer_i, 1)
    #print(fitting_mesh_dancer_i)
    fitting_mesh_dancer_i.compute_vertex_normals()
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_{i+offset:03}.obj', fitting_mesh_dancer_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    #o3d.visualization.draw_geometries([fitting_mesh_dancer_i])

[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.


In [4]:
def compute_D1_psnr(original_mesh, decoded_mesh):
    
    original_vertices = np.array(original_mesh.vertices)
    #original_vertices = normalize_vertices(original_vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    #decoded_vertices = normalize_vertices(decoded_vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.linalg.norm(original_vertices[i] - decoded_vertices[index]))
    MSE = MSE / len(original_vertices)
    #print(MSE)
    aabb = pcd_original.get_axis_aligned_bounding_box()
    min_bound = aabb.get_min_bound()

    max_bound = aabb.get_max_bound()

    signal_peak = np.linalg.norm(max_bound - min_bound)
    #print(signal_peak)
    psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)
    print(psnr)
    return psnr

In [40]:
def compute_D2_psnr(original_mesh, decoded_mesh):
    decoded_mesh.compute_vertex_normals()
    
    original_vertices = np.array(original_mesh.vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_decoded.normals = o3d.utility.Vector3dVector(decoded_mesh.vertex_normals)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.dot((original_vertices[i] - decoded_vertices[index])[0], np.array(pcd_decoded.normals)[index][0]))
    MSE = MSE / len(original_vertices)
    
    aabb = pcd_original.get_axis_aligned_bounding_box()
    min_bound = aabb.get_min_bound()

    max_bound = aabb.get_max_bound()

    signal_peak = np.linalg.norm(max_bound - min_bound)
    psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)
    
    return psnr

In [44]:
GoF = 5
for i in range(0, 5):
    offset = 5
    reconstruct_dancer_i = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_{i+offset:03}.obj', enable_post_processing=False)
    original_dancer_i = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0{i+offset:03}.obj')
    print(reconstruct_dancer_i)
    print(original_dancer_i)
    fitting_mesh_dancer_i = subdivide_surface_fitting(reconstruct_dancer_i, original_dancer_i, 1)
    fitting_mesh_dancer_i.compute_vertex_normals()
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_{i+offset:03}.obj', fitting_mesh_dancer_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    #o3d.visualization.draw_geometries([fitting_mesh_dancer_i])

TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 20788 points and 39380 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 20833 points and 39411 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 20658 points and 39373 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 20778 points and 39392 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 20887 points and 39356 triangles.
TriangleMesh with 19996 points an

In [27]:
input_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/reference_mesh.obj'
input_path2 = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/order_reference_mesh.obj'
output_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/encode.drc'

## get error

In [5]:
reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj')
print(reference_mesh)
decimated_reference_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(reference_mesh, 10000, boundary_weight= 8000)
print(decimated_reference_mesh)
decimated_reference_mesh_vertices = np.array(decimated_reference_mesh.vertices)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', decimated_reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
loaded_decimated_reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
print(loaded_decimated_reference_mesh)
loaded_decimated_reference_mesh_vertices = np.array(loaded_decimated_reference_mesh.vertices)

TriangleMesh with 20002 points and 40000 triangles.
TriangleMesh with 5002 points and 10000 triangles.
TriangleMesh with 4996 points and 10000 triangles.


In [6]:
subdivided_decimated_reference_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_reference_mesh, number_of_iterations=1)
print(subdivided_decimated_reference_mesh)
subdivided_decimated_reference_mesh_vertices = np.array(subdivided_decimated_reference_mesh.vertices)

TriangleMesh with 20002 points and 40000 triangles.


## get displacements

In [7]:
decimated_reference_mesh = loaded_decimated_reference_mesh
print(decimated_reference_mesh)

TriangleMesh with 4996 points and 10000 triangles.


In [8]:
dataset = 'Dancer'
GoF = 5

In [9]:
for i in range(0, GoF):
    offset = 5
    reconstruct_dancer_i = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_{i+offset:03}.obj')
    original_dancer_i = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0{i+offset:03}.obj')

    fitting_mesh_dancer_i = subdivide_surface_fitting(reconstruct_dancer_i, original_dancer_i, 1)
    print(fitting_mesh_dancer_i)
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_{i+offset:03}.obj', fitting_mesh_dancer_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    #o3d.visualization.draw_geometries([fitting_mesh_dancer_i])

TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.


In [56]:
subdivided_decimated_reference_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_reference_mesh, number_of_iterations=1)
print(subdivided_decimated_reference_mesh)
subdivided_decimated_reference_mesh_vertices = np.array(subdivided_decimated_reference_mesh.vertices)
subdivided_decimated_reference_mesh_vertices

TriangleMesh with 19996 points and 40000 triangles.


array([[ 0.0758296 ,  0.911741  , -0.472649  ],
       [ 0.0902357 ,  0.911741  , -0.48194   ],
       [ 0.0901355 ,  0.903519  , -0.477867  ],
       ...,
       [ 0.1807615 , -0.4135865 ,  0.306595  ],
       [-0.0433    ,  0.7934835 ,  0.3187445 ],
       [-0.05850105,  0.398496  ,  0.316869  ]])

In [57]:
displacements = []
for i in range(0, GoF):
    offset = 5
    fitting_mesh_dancer_i = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_{i+offset:03}.obj')
    print(fitting_mesh_dancer_i, subdivided_decimated_reference_mesh)
    fitting_mesh_vertices = np.array(fitting_mesh_dancer_i.vertices)
    displacement_i = subdivided_decimated_reference_mesh_vertices - fitting_mesh_vertices
    
    np.savetxt(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/displacements_{dataset}_{i+offset:03}.txt', displacement_i, fmt='%8f')
    displacements.append(displacement_i)

TriangleMesh with 19996 points and 40000 triangles. TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles. TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles. TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles. TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles. TriangleMesh with 19996 points and 40000 triangles.


In [10]:
input_reference_mesh_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj'
input_decimated_reference_mesh = read_triangle_mesh_with_trimesh(input_reference_mesh_path, enable_post_processing=False)
np.array(input_decimated_reference_mesh.vertices)

array([[ 0.0758296,  0.911741 , -0.472649 ],
       [ 0.0902357,  0.911741 , -0.48194  ],
       [ 0.0901355,  0.903519 , -0.477867 ],
       ...,
       [ 0.0342721,  0.614468 ,  0.0390702],
       [ 0.289648 , -0.456912 ,  0.0853124],
       [-0.201152 ,  0.548629 ,  0.0756965]])

In [11]:
output_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/encoded_decimated_reference_mesh.drc'
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_encoder',
                            '-i', input_reference_mesh_path,
                            '-o', output_path,
                            '-qp', str('14'),
                            '-cl', '7'
                            ], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

Encoder options:
  Compression level = 7
  Positions: Quantization = 14 bits

Encoded mesh saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/encoded_decimated_reference_mesh.drc (47 ms to encode).

Encoded size = 18111 bytes

For better compression, increase the compression level up to '-cl 10' .





In [12]:
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_decoder',
                            '-i', output_path,
                            '-o', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decode_decimated_reference_mesh.obj'
                            ], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

Decoded geometry saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decode_decimated_reference_mesh.obj (12 ms to decode)




In [5]:
pcd = o3d.geometry.PointCloud()
points = displacement
pcd.points = o3d.utility.Vector3dVector(points)
print(pcd)
o3d.io.write_point_cloud(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.ply', pcd)

PointCloud with 19996 points.


True

In [124]:
displacement = np.loadtxt(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/displacements_{dataset}_005')
pcd = o3d.geometry.PointCloud()
points = displacement
pcd.points = o3d.utility.Vector3dVector(points)
print(pcd)
o3d.io.write_point_cloud(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.ply', pcd)
points=np.asarray(pcd.points)
dtype = o3d.core.float32
p_tensor = o3d.core.Tensor(points, dtype=dtype)
pc = o3d.t.geometry.PointCloud(p_tensor)
o3d.t.io.write_point_cloud(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis_32.ply', pc, write_ascii=True)

True

In [35]:
qp = 12
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_encoder',
                            #r'G:\Github\draco\buildforSequenceEncoding\Debug\draco_encoder',
                            '-point_cloud',
                            '-i', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis_32.ply',
                            '-o', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.drc',
                            '-qp', str(qp),
                            '-cl', '10'
                            ], capture_output=True, text=True)
print(result.stdout)

result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_encoder',
                            #r'G:\Github\draco\buildforSequenceEncoding\Debug\draco_encoder',
                            '-point_cloud',
                            '-i', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis_32.ply',
                            '-o', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis_0.drc',
                            '-qp', str(qp),
                            '-cl', '0'
                            ], capture_output=True, text=True)
print(result.stdout)

Encoder options:
  Compression level = 10
  Positions: Quantization = 12 bits

Encoded point cloud saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.drc (27 ms to encode).

Encoded size = 36591 bytes


Encoder options:
  Compression level = 0
  Positions: Quantization = 12 bits

Encoded point cloud saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis_0.drc (7 ms to encode).

Encoded size = 63825 bytes

For better compression, increase the compression level up to '-cl 10' .




In [36]:
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_decoder',
                            '-i', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis_0.drc',
                            '-o', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decoded_displacements.ply'
                            ], capture_output=True, text=True)
print(result.stdout)

Decoded geometry saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decoded_displacements.ply (4 ms to decode)



In [15]:
def calculate_bitrate(file_size, duration):
    return file_size * 8 / duration

In [16]:
number_frames = 5
frame_rate = 30

In [37]:
total_size = 0
displacement_file_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.drc'
displacement_file_size = os.path.getsize(displacement_file_path)
total_size += displacement_file_size * number_frames
reference_mesh_file_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/1.drc'
reference_mesh_file_size = os.path.getsize(reference_mesh_file_path)
total_size += reference_mesh_file_size
total_duration = number_frames / frame_rate
overall_bitrate = calculate_bitrate(total_size, total_duration)

print(f"Total Size of {number_frames} DRC Files: {total_size} bytes")
print(f"Overall Bitrate: {overall_bitrate} bits per second")

bitrate_kbps = overall_bitrate / 1000
bitrate_mbps = overall_bitrate / 1000000


print(f"Overall Bitrate: {bitrate_kbps:.2f} Kbps")
print(f"Overall Bitrate: {bitrate_mbps:.2f} Mbps")

Total Size of 5 DRC Files: 201066 bytes
Overall Bitrate: 9651168.0 bits per second
Overall Bitrate: 9651.17 Kbps
Overall Bitrate: 9.65 Mbps


In [38]:
original_displacements = np.loadtxt(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/displacements_{dataset}_005.txt')
decoded_displacements = o3d.io.read_point_cloud(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decoded_displacements.ply')
dis_ply = o3d.io.read_point_cloud(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decoded_displacements.ply')
displacement = np.array(dis_ply.points)
displacement

array([[ 0.20801824,  0.02408319,  0.00548263],
       [ 0.20581117,  0.02408319,  0.00683141],
       [ 0.20581117,  0.02445104,  0.00781234],
       ...,
       [ 0.02090642,  0.00311588,  0.00413386],
       [-0.00373936, -0.00951356,  0.00572786],
       [ 0.00472113,  0.00188972, -0.00273263]])

In [41]:
decode_decimated_reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decode_decimated_reference_mesh.obj', enable_post_processing=False)
print(decode_decimated_reference_mesh)
np.array(decode_decimated_reference_mesh.vertices)
subdivided_decoded_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decode_decimated_reference_mesh, number_of_iterations=1)
print(subdivided_decoded_mesh)
input_decimated_reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(input_decimated_reference_mesh, number_of_iterations=1)
print(subdivided_mesh)
original_dancer = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0005.obj')
print(original_dancer)
decoded_mesh_vertices = np.array(decode_decimated_reference_mesh.vertices)
subdivided_decoded_mesh_vertices = np.array(subdivided_decoded_mesh.vertices)


displacement = np.array(decoded_displacements.points)

dis_indexer = o3d.geometry.PointCloud()
dis_indexer.points = o3d.utility.Vector3dVector(original_displacements)
dis_tree = o3d.geometry.KDTreeFlann(dis_indexer)

pcd_indexer = o3d.geometry.PointCloud()
pcd_indexer.points = o3d.utility.Vector3dVector(subdivided_mesh.vertices)
pcd_tree = o3d.geometry.KDTreeFlann(pcd_indexer)

reordered_vertices = deepcopy(subdivided_decoded_mesh_vertices)

for i in range(0, len(subdivided_decoded_mesh_vertices)):
    [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_decoded_mesh_vertices[i], 1)
    [j, dis_index, _] = dis_tree.search_knn_vector_3d(original_displacements[index[0]], 1)
    #print(displacement[dis_index], original_displacements[index[0]])
    reordered_vertices[i] -= displacement[dis_index[0]]
    

subdivided_decoded_mesh.vertices = o3d.utility.Vector3dVector(reordered_vertices)
subdivided_decoded_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([subdivided_decoded_mesh])
d1 = max(compute_D1_psnr(original_dancer, subdivided_decoded_mesh), compute_D1_psnr(subdivided_decoded_mesh, original_dancer))
print("D1:", d1)
d2 = max(compute_D2_psnr(original_dancer, subdivided_decoded_mesh), compute_D2_psnr(subdivided_decoded_mesh, original_dancer))
print("D2:", d2)

TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 20788 points and 39380 triangles.
54.892510781309696
88.9873418319456
D1: 88.9873418319456
D2: 93.79743987464224
